In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from multiprocessing import Pool #Multithreading
import time 
import random


#Import "att532.tsp" file to read the cities (http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/)
def read_cities(filename):
    with open(filename, 'r') as f:
        # Skip the header lines
        for i in range(6):
            next(f)
        
        # Extract the coordinates and build the tuples
        cities = []
        for line in f:
            if line.startswith('EOF'):
                break
            values = line.split()
            if len(values) >= 3:
                x, y = values[1], values[2]
                cities.append((float(x), float(y)))
    return cities
def ant_colony_stp(cities, start_city, num_ants, alpha, beta, q0, pheromone_decay, num_iterations):
    # Step 2: Initialize the ants
    ants = [[start_city] for i in range(num_ants)]

    # Step 3: Define the pheromone matrix
    pheromone = np.ones((len(cities), len(cities))) * 0.1

    # Step 4: Define the heuristic information matrix
    heuristic = np.zeros((len(cities), len(cities)))
    for i in range(len(cities)):
        for j in range(len(cities)):
            heuristic[i][j] = np.sqrt((cities[i][0]-cities[j][0])**2 + (cities[i][1]-cities[j][1])**2)

    # Step 5: Define the ant's movement rules
    for iteration in range(num_iterations):
        for i in range(num_ants):
            for j in range(len(cities)-1):
                current_city = ants[i][-1]
                unvisited_cities = [city for city in range(len(cities)) if city not in ants[i]]
                if not unvisited_cities:
                    break
                probabilities = []
                for city in unvisited_cities:
                    pheromone_level = pheromone[current_city][city]
                    heuristic_info = heuristic[current_city][city]
                    probability = (pheromone_level**alpha) * (heuristic_info**beta)
                    probabilities.append(probability)
                if not probabilities:
                    next_city = ants[i][0]
                elif random.random() < q0:
                    next_city = unvisited_cities[np.argmax(probabilities)]
                else:
                    total_prob = sum(probabilities)
                    probabilities = [p/total_prob for p in probabilities]
                    next_city = np.random.choice(unvisited_cities, p=probabilities)
                ants[i].append(next_city)

            # Step 6: Update the pheromone levels
            path_length = sum([heuristic[ants[i][j]][ants[i][j+1]] for j in range(len(cities)-1)])
            for j in range(len(cities)-1):
                pheromone[ants[i][j]][ants[i][j+1]] += 1/path_length
            pheromone *= (1-pheromone_decay)

        # Step 7: Check for convergence
        # You can check for convergence by monitoring the best path found so far and stopping when it doesn't improve for a certain number of iterations.
        # I'll leave the implementation of this part up to you.

    # Once you have converged, you can return the best path found by the ants.
    best_path_indices = ants[np.argmin([sum([heuristic[ants[i][j]][ants[i][j+1]] for j in range(len(cities)-1)]) for i in range(num_ants)])]
    best_path = [cities[i] for i in best_path_indices]
    total_distance = sum([heuristic[best_path_indices[j]][best_path_indices[j+1]] for j in range(len(cities)-1)])
    return best_path, total_distance
import time
cities = read_cities('/kaggle/input/d1291-tsplib-dataset/d1291.tsp')
start_city = 0
num_ants = 10
alpha = 1
beta = 2
q0 = 0.9
pheromone_decay = 0.1
num_iterations = 100
start_time = time.time_ns() // 1000 
best_path, total_distance = ant_colony_stp(cities, start_city, num_ants, alpha, beta, q0, pheromone_decay, num_iterations)
end_time = time.time_ns() // 1000 
elapsed_time = end_time - start_time
print("WITHOUT PARALLELISM AND 1291 CITIES")
print("Elapsed time: {} microseconds".format(elapsed_time))
#print("Best path:", best_path)
print("Total distance:", total_distance)

WITHOUT PARALLELISM AND 1291 CITIES
Elapsed time: 187447940 microseconds
Total distance: 2430814.2695076275
